# 10 Lowest EQAO Ratings of Elementary Schools in Toronto Distric Schools (TDSB) & Toronto Catholic Distric School Board (TCDSB)

In [1]:
%matplotlib notebook

In [2]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import json
from pprint import pprint
from config import gkey

In [3]:
toronto_schools = pd.read_csv("output_data/toronto_schools.csv")
catholic_schools = pd.read_csv("output_data/toronto_catholic_schools.csv")
home_prices = pd.read_csv("output_data/toronto_api_postal_combined and filtered with price range 100K to 10M.csv")

#### Average Home Prices by FSA

In [4]:
home_prices.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,19,Malvern,114900,43.803925,-79.216507,#105 - 8 Silverbell Grve,Toronto,M1B 2L7,M1B
1,46,Malvern,125000,43.805328,-79.220230,#310 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
2,47,Malvern,125000,43.805328,-79.220230,#101 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
3,74,Malvern,139900,43.805328,-79.220230,#405 - 99 BLACKWELL AVE,Toronto,M1B 3R5,M1B
4,79,Malvern,143000,43.803925,-79.216507,#504 - 8 SILVER BELL GRVE,Toronto,M1B 2L7,M1B


In [5]:
prices_fsa = home_prices[["Price ($)", "lat", "lng", "FSA"]]
prices_fsa.head()

,Price ($),lat,lng,FSA
0,114900,43.803925,-79.216507,M1B
1,125000,43.805328,-79.220230,M1B
2,125000,43.805328,-79.220230,M1B
3,139900,43.805328,-79.220230,M1B
4,143000,43.803925,-79.216507,M1B


In [6]:
avg_prices_fsa = prices_fsa.groupby(["FSA"]).mean()
avg_prices_fsa = avg_prices_fsa.rename(columns={"Price ($)":"AVG HOME PRICES"})
avg_prices_fsa["AVG HOME PRICES"] = avg_prices_fsa["AVG HOME PRICES"].round()
avg_prices_fsa.head()

,AVG HOME PRICES,lat,lng
FSA,,,
M1B,516127.0,43.803708,-79.217831
M1C,945254.0,43.789515,-79.159737
M1E,619287.0,43.772357,-79.193029
M1G,655781.0,43.770924,-79.219387
M1H,497626.0,43.775830,-79.246489


#### 10 Lowest TDSB EQAO Ratings

In [7]:
toronto_schools.sort_values("RATINGS", ascending=True).head(10)

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANKING (OUT OF 3046),RATINGS
197,F H Miller,300 Caledonia Road,York,M6E 4T5,M6E,3011,0.7
161,First Nations School of Toronto,935 Dundas St E,Toronto,M4M 1R4,M4M,2985,1.6
96,Highland Heights,35 Glendower Circuit,Agincourt,M1T 2Z3,M1T,2976,1.8
124,R J Lang,227 Drewry Ave,Toronto,M2M 1E3,M2M,2971,1.9
55,Ionview,90 Ionview Road,Scarborough,M1K 2Z9,M1K,2966,2.0
23,Eastview,20 Waldock Street,West Hill,M1E 2E5,M1E,2951,2.3
216,Parkdale,78 Seaforth Avenue,Toronto,M6K 3L2,M6K,2904,2.8
112,Chester Le,201 Chester Le Boulevard,Agincourt,M1W 2K7,M1W,2848,3.2
31,Joseph Brant,270 Manse Road,West Hill,M1E 3V4,M1E,2848,3.2
27,West Hill,299 Morningside Avenue,West Hill,M1E 3G1,M1E,2848,3.2


In [8]:
lowest_ratings_tdsb = toronto_schools[toronto_schools["RATINGS"] < 3.3]
lowest_ratings_tdsb

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANKING (OUT OF 3046),RATINGS
23,Eastview,20 Waldock Street,West Hill,M1E 2E5,M1E,2951,2.3
27,West Hill,299 Morningside Avenue,West Hill,M1E 3G1,M1E,2848,3.2
31,Joseph Brant,270 Manse Road,West Hill,M1E 3V4,M1E,2848,3.2
55,Ionview,90 Ionview Road,Scarborough,M1K 2Z9,M1K,2966,2.0
96,Highland Heights,35 Glendower Circuit,Agincourt,M1T 2Z3,M1T,2976,1.8
112,Chester Le,201 Chester Le Boulevard,Agincourt,M1W 2K7,M1W,2848,3.2
124,R J Lang,227 Drewry Ave,Toronto,M2M 1E3,M2M,2971,1.9
161,First Nations School of Toronto,935 Dundas St E,Toronto,M4M 1R4,M4M,2985,1.6
197,F H Miller,300 Caledonia Road,York,M6E 4T5,M6E,3011,0.7
216,Parkdale,78 Seaforth Avenue,Toronto,M6K 3L2,M6K,2904,2.8


In [9]:
lowest_tdsb = lowest_ratings_tdsb[["SCHOOL NAME", "ADDRESS", "POSTAL CODE", "FSA", "RATINGS"]]
lowest_tdsb

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS
23,Eastview,20 Waldock Street,M1E 2E5,M1E,2.3
27,West Hill,299 Morningside Avenue,M1E 3G1,M1E,3.2
31,Joseph Brant,270 Manse Road,M1E 3V4,M1E,3.2
55,Ionview,90 Ionview Road,M1K 2Z9,M1K,2.0
96,Highland Heights,35 Glendower Circuit,M1T 2Z3,M1T,1.8
112,Chester Le,201 Chester Le Boulevard,M1W 2K7,M1W,3.2
124,R J Lang,227 Drewry Ave,M2M 1E3,M2M,1.9
161,First Nations School of Toronto,935 Dundas St E,M4M 1R4,M4M,1.6
197,F H Miller,300 Caledonia Road,M6E 4T5,M6E,0.7
216,Parkdale,78 Seaforth Avenue,M6K 3L2,M6K,2.8


In [10]:
lowest_tdsb_homes = pd.merge(lowest_tdsb, avg_prices_fsa, on="FSA")
lowest_tdsb_homes

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Eastview,20 Waldock Street,M1E 2E5,M1E,2.3,619287.0,43.772357,-79.193029
1,West Hill,299 Morningside Avenue,M1E 3G1,M1E,3.2,619287.0,43.772357,-79.193029
2,Joseph Brant,270 Manse Road,M1E 3V4,M1E,3.2,619287.0,43.772357,-79.193029
3,Ionview,90 Ionview Road,M1K 2Z9,M1K,2.0,632037.0,43.730115,-79.263112
4,Highland Heights,35 Glendower Circuit,M1T 2Z3,M1T,1.8,540841.0,43.785104,-79.298052
5,Chester Le,201 Chester Le Boulevard,M1W 2K7,M1W,3.2,519808.0,43.804687,-79.319888
6,R J Lang,227 Drewry Ave,M2M 1E3,M2M,1.9,1103203.0,43.786387,-79.413706
7,First Nations School of Toronto,935 Dundas St E,M4M 1R4,M4M,1.6,822616.0,43.663904,-79.341013
8,F H Miller,300 Caledonia Road,M6E 4T5,M6E,0.7,948624.0,43.687255,-79.449567
9,Parkdale,78 Seaforth Avenue,M6K 3L2,M6K,2.8,515311.0,43.639351,-79.424010


#### 10 Lowest TCDSB EQAO Ratings

In [11]:
catholic_schools.sort_values("RATINGS", ascending=True).head(10)

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANK (OUT OF 3046),RATINGS
39,Epiphany of our Lord,3150 Pharmacy Avenue,Toronto,M1W 1J5,M1W,2932,2.5
149,St Dorothy,155 John Garland Blvd,Toronto,M9V 1N7,M9V,2932,2.5
136,St Roch,174 Duncanwoods Drive,Toronto,M9L 2E3,M9L,2932,2.5
70,St Charles Garnier,20 Stong Court,Toronto,M3N 2X9,M3N,2904,2.8
107,Holy Family,141 Close Avenue,Toronto,M6K 2V6,M6K,2875,3.0
154,St Stephen,55 Golfdown Drive,Toronto,M9W 2H8,M9W,2835,3.3
113,Our Lady of Victory,70 Guestville Avenue,Toronto,M6N 4N3,M6N,2815,3.4
79,St Joseph,176 Leslie Street,Toronto,M4M 3C7,M4M,2765,3.6
97,Stella Maris,31 Ascot Avenue,Toronto,M6E 1E6,M6E,2735,3.7
114,Santa Maria,25 Avon Avenue,Toronto,M6N 4X8,M6N,2735,3.7


In [12]:
lowest_ratings_tcdsb = catholic_schools[catholic_schools["RATINGS"] < 3.8]
lowest_ratings_tcdsb

,SCHOOL NAME,ADDRESS,CITY,POSTAL CODE,FSA,RANK (OUT OF 3046),RATINGS
39,Epiphany of our Lord,3150 Pharmacy Avenue,Toronto,M1W 1J5,M1W,2932,2.5
70,St Charles Garnier,20 Stong Court,Toronto,M3N 2X9,M3N,2904,2.8
79,St Joseph,176 Leslie Street,Toronto,M4M 3C7,M4M,2765,3.6
97,Stella Maris,31 Ascot Avenue,Toronto,M6E 1E6,M6E,2735,3.7
107,Holy Family,141 Close Avenue,Toronto,M6K 2V6,M6K,2875,3.0
113,Our Lady of Victory,70 Guestville Avenue,Toronto,M6N 4N3,M6N,2815,3.4
114,Santa Maria,25 Avon Avenue,Toronto,M6N 4X8,M6N,2735,3.7
136,St Roch,174 Duncanwoods Drive,Toronto,M9L 2E3,M9L,2932,2.5
149,St Dorothy,155 John Garland Blvd,Toronto,M9V 1N7,M9V,2932,2.5
154,St Stephen,55 Golfdown Drive,Toronto,M9W 2H8,M9W,2835,3.3


In [13]:
lowest_tcdsb = lowest_ratings_tcdsb[["SCHOOL NAME", "ADDRESS", "POSTAL CODE", "FSA", "RATINGS"]]
lowest_tcdsb

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS
39,Epiphany of our Lord,3150 Pharmacy Avenue,M1W 1J5,M1W,2.5
70,St Charles Garnier,20 Stong Court,M3N 2X9,M3N,2.8
79,St Joseph,176 Leslie Street,M4M 3C7,M4M,3.6
97,Stella Maris,31 Ascot Avenue,M6E 1E6,M6E,3.7
107,Holy Family,141 Close Avenue,M6K 2V6,M6K,3.0
113,Our Lady of Victory,70 Guestville Avenue,M6N 4N3,M6N,3.4
114,Santa Maria,25 Avon Avenue,M6N 4X8,M6N,3.7
136,St Roch,174 Duncanwoods Drive,M9L 2E3,M9L,2.5
149,St Dorothy,155 John Garland Blvd,M9V 1N7,M9V,2.5
154,St Stephen,55 Golfdown Drive,M9W 2H8,M9W,3.3


In [14]:
lowest_tcdsb_homes = pd.merge(lowest_tcdsb, avg_prices_fsa, on="FSA")
lowest_tcdsb_homes

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Epiphany of our Lord,3150 Pharmacy Avenue,M1W 1J5,M1W,2.5,519808.0,43.804687,-79.319888
1,St Charles Garnier,20 Stong Court,M3N 2X9,M3N,2.8,389532.0,43.751457,-79.517602
2,St Joseph,176 Leslie Street,M4M 3C7,M4M,3.6,822616.0,43.663904,-79.341013
3,Stella Maris,31 Ascot Avenue,M6E 1E6,M6E,3.7,948624.0,43.687255,-79.449567
4,Holy Family,141 Close Avenue,M6K 2V6,M6K,3.0,515311.0,43.639351,-79.424010
5,Our Lady of Victory,70 Guestville Avenue,M6N 4N3,M6N,3.4,641388.0,43.675521,-79.480776
6,Santa Maria,25 Avon Avenue,M6N 4X8,M6N,3.7,641388.0,43.675521,-79.480776
7,St Roch,174 Duncanwoods Drive,M9L 2E3,M9L,2.5,626995.0,43.755605,-79.565699
8,St Dorothy,155 John Garland Blvd,M9V 1N7,M9V,2.5,490494.0,43.744364,-79.588308
9,St Stephen,55 Golfdown Drive,M9W 2H8,M9W,3.3,490617.0,43.725549,-79.580242


### 10 Lowest EQAO Rating for TDSB & TCDSB with Home Prices

In [15]:
lowest_schools = pd.concat([lowest_tdsb_homes, lowest_tdsb_homes])
lowest_schools

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,lat,lng
0,Eastview,20 Waldock Street,M1E 2E5,M1E,2.3,619287.0,43.772357,-79.193029
1,West Hill,299 Morningside Avenue,M1E 3G1,M1E,3.2,619287.0,43.772357,-79.193029
2,Joseph Brant,270 Manse Road,M1E 3V4,M1E,3.2,619287.0,43.772357,-79.193029
3,Ionview,90 Ionview Road,M1K 2Z9,M1K,2.0,632037.0,43.730115,-79.263112
4,Highland Heights,35 Glendower Circuit,M1T 2Z3,M1T,1.8,540841.0,43.785104,-79.298052
5,Chester Le,201 Chester Le Boulevard,M1W 2K7,M1W,3.2,519808.0,43.804687,-79.319888
6,R J Lang,227 Drewry Ave,M2M 1E3,M2M,1.9,1103203.0,43.786387,-79.413706
7,First Nations School of Toronto,935 Dundas St E,M4M 1R4,M4M,1.6,822616.0,43.663904,-79.341013
8,F H Miller,300 Caledonia Road,M6E 4T5,M6E,0.7,948624.0,43.687255,-79.449567
9,Parkdale,78 Seaforth Avenue,M6K 3L2,M6K,2.8,515311.0,43.639351,-79.424010


In [16]:
lowest_schools = lowest_schools.rename(columns={"lat":"HOME_LAT", "lng":"HOME_LNG"})
lowest_schools["SCHOOL_LAT"] = ""
lowest_schools["SCHOOL_LNG"] = ""
lowest_schools.head()

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,HOME_LAT,HOME_LNG,SCHOOL_LAT,SCHOOL_LNG
0,Eastview,20 Waldock Street,M1E 2E5,M1E,2.3,619287.0,43.772357,-79.193029,,
1,West Hill,299 Morningside Avenue,M1E 3G1,M1E,3.2,619287.0,43.772357,-79.193029,,
2,Joseph Brant,270 Manse Road,M1E 3V4,M1E,3.2,619287.0,43.772357,-79.193029,,
3,Ionview,90 Ionview Road,M1K 2Z9,M1K,2.0,632037.0,43.730115,-79.263112,,
4,Highland Heights,35 Glendower Circuit,M1T 2Z3,M1T,1.8,540841.0,43.785104,-79.298052,,


In [17]:
params = {"key": gkey}

for index, row in lowest_schools.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    address = row["ADDRESS"]

    params['address'] = f"{address},+Toronto,+ON"

    schools_lat_lng = requests.get(base_url, params=params)
    
    schools_lat_lng = schools_lat_lng.json()

    lowest_schools.loc[index, "SCHOOL_LAT"] = schools_lat_lng["results"][0]["geometry"]["location"]["lat"]
    lowest_schools.loc[index, "SCHOOL_LNG"] = schools_lat_lng["results"][0]["geometry"]["location"]["lng"]

lowest_schools

,SCHOOL NAME,ADDRESS,POSTAL CODE,FSA,RATINGS,AVG HOME PRICES,HOME_LAT,HOME_LNG,SCHOOL_LAT,SCHOOL_LNG
0,Eastview,20 Waldock Street,M1E 2E5,M1E,2.3,619287.0,43.772357,-79.193029,43.759,-79.1914
1,West Hill,299 Morningside Avenue,M1E 3G1,M1E,3.2,619287.0,43.772357,-79.193029,43.7728,-79.1867
2,Joseph Brant,270 Manse Road,M1E 3V4,M1E,3.2,619287.0,43.772357,-79.193029,43.7669,-79.1756
3,Ionview,90 Ionview Road,M1K 2Z9,M1K,2.0,632037.0,43.730115,-79.263112,43.7375,-79.2743
4,Highland Heights,35 Glendower Circuit,M1T 2Z3,M1T,1.8,540841.0,43.785104,-79.298052,43.7945,-79.3003
5,Chester Le,201 Chester Le Boulevard,M1W 2K7,M1W,3.2,519808.0,43.804687,-79.319888,43.799,-79.3315
6,R J Lang,227 Drewry Ave,M2M 1E3,M2M,1.9,1103203.0,43.786387,-79.413706,43.7838,-79.4266
7,First Nations School of Toronto,935 Dundas St E,M4M 1R4,M4M,1.6,822616.0,43.663904,-79.341013,43.6621,-79.3487
8,F H Miller,300 Caledonia Road,M6E 4T5,M6E,0.7,948624.0,43.687255,-79.449567,43.6833,-79.4583
9,Parkdale,78 Seaforth Avenue,M6K 3L2,M6K,2.8,515311.0,43.639351,-79.424010,43.643,-79.437


In [24]:
lowest_schools.to_csv("output_data/lowest_schools_yd.csv")